## Using wine dataset to apply machine learning in order to find the wine rank

#### Impot needed library

In [1]:
# need library
import numpy as np
import tensorflow as tf
# warning remove
if type(tf.contrib) != type(tf):
    tf.contrib._warning = None
# 딥러닝 구동에 필요한 케라스 함수
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [5]:
import pandas as pd

df_pre = pd.read_csv('dataset/wine_data.csv', header=None)
df = df_pre.sample(frac=1) # 원본 데이터의 100% 추출
df.head(10)

,0,1,2,3,4,5,6,7,8,9,10,11,12
4481,7.0,0.24,0.30,6.70,0.039,37.0,125.0,0.99436,3.20,0.39,9.9,0,5
3573,7.5,0.26,0.26,18.35,0.084,33.0,139.0,1.00110,3.17,0.39,8.8,0,5
3175,6.2,0.27,0.49,1.40,0.050,20.0,74.0,0.99310,3.32,0.44,9.8,0,6
2963,7.6,0.19,0.41,1.10,0.040,38.0,143.0,0.99070,2.92,0.42,11.4,0,5
5541,7.2,0.23,0.39,1.50,0.053,26.0,106.0,0.99166,3.18,0.47,11.1,0,6
3690,7.2,0.24,0.19,7.70,0.045,53.0,176.0,0.99580,3.17,0.38,9.5,0,5
2230,7.0,0.23,0.42,18.05,0.050,35.0,144.0,0.99990,3.22,0.42,8.8,0,5
737,8.2,0.59,0.00,2.50,0.093,19.0,58.0,1.00020,3.50,0.65,9.3,1,6
4716,6.5,0.23,0.39,1.90,0.036,41.0,98.0,0.99000,3.19,0.43,11.9,0,7
1755,7.4,0.24,0.42,14.00,0.066,48.0,198.0,0.99790,2.89,0.42,8.9,0,6


In [7]:
# X : explanatory variables, Y : response variable (와인의 10단계 등급)
dataset = df.values
X = dataset[:,0:12]
Y_obj = dataset[:,12]

#### One-hot encoding

In [8]:
from keras.utils import np_utils

Y = np_utils.to_categorical(Y_obj,11)

In [9]:
Y[0]

array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.], dtype=float32)

#### modeling (Deep-learning), update
- 은닉노드의 개수와 활성화 함수를 다르게 적용하여 4가지의 딥러닝 모델 구축 후 성능 비교
###### model 1
- input 12
- 4 hidden layer , 30 15 10 * 11 node
- activation function : relu -> relu -> softplus -> softmax

In [10]:
from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping

import pandas as pd
import numpy as np
import os
import tensorflow as tf

seed = 0
np.random.seed(seed)
tf.set_random_seed(seed)

# modeling
model = Sequential()
model.add(Dense(30, input_dim=12, activation='relu'))
model.add(Dense(15, activation='relu'))
model.add(Dense(10, activation='softplus'))
model.add(Dense(11, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

W0819 15:20:08.705277   516 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0819 15:20:08.707277   516 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0819 15:20:08.713277   516 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0819 15:20:08.771281   516 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0819 15:20:08.788282   516 deprecation_wrapper.py:119] From C:\ProgramData\Anac

In [11]:
# model update
MODEL_DIR = 'model1/'
if not os.path.exists(MODEL_DIR):
    os.mkdir(MODEL_DIR)

modelpath = "model1/{epoch:02d}-{val_loss:.4f}.hdf5"
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', verbose=1, save_best_only=True)
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=100)

history1 = model.fit(X, Y, validation_split=0.3, epochs=3000, batch_size=500, callbacks=[early_stopping_callback, checkpointer])

W0819 15:21:05.824104   516 deprecation.py:323] From C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0819 15:21:05.865107   516 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 4547 samples, validate on 1950 samples
Epoch 1/3000
4547/4547 [==============================] - 0s 56us/step - loss: 6.6996 - acc: 0.1394 - val_loss: 3.7274 - val_acc: 0.0985

Epoch 00001: val_loss improved from inf to 3.72745, saving model to model1/01-3.7274.hdf5
Epoch 2/3000
4547/4547 [==============================] - 0s 3us/step - loss: 2.7737 - acc: 0.1821 - val_loss: 2.2266 - val_acc: 0.3564

Epoch 00002: val_loss improved from 3.72745 to 2.22656, saving model to model1/02-2.2266.hdf5
Epoch 3/3000
4547/4547 [==============================] - 0s 3us/step - loss: 2.2072 - acc: 0.3734 - val_loss: 2.1628 - val_acc: 0.4108

Epoch 00003: val_loss improved from 2.22656 to 2.16285, saving model to model1/03-2.1628.hdf5
Epoch 4/3000
4547/4547 [==============================] - 0s 3us/step - loss: 2.0481 - acc: 0.4108 - val_loss: 1.8994 - val_acc: 0.4241

Epoch 00004: val_loss improved from 2.16285 to 1.89941, saving model to model1/04-1.8994.hdf5
Epoch 5/3000
4547/4547 [=======


Epoch 00036: val_loss improved from 1.23952 to 1.23604, saving model to model1/36-1.2360.hdf5
Epoch 37/3000
4547/4547 [==============================] - 0s 3us/step - loss: 1.2348 - acc: 0.4781 - val_loss: 1.2341 - val_acc: 0.4600

Epoch 00037: val_loss improved from 1.23604 to 1.23412, saving model to model1/37-1.2341.hdf5
Epoch 38/3000
4547/4547 [==============================] - 0s 3us/step - loss: 1.2339 - acc: 0.4717 - val_loss: 1.2339 - val_acc: 0.4595

Epoch 00038: val_loss improved from 1.23412 to 1.23389, saving model to model1/38-1.2339.hdf5
Epoch 39/3000
4547/4547 [==============================] - 0s 4us/step - loss: 1.2312 - acc: 0.4733 - val_loss: 1.2345 - val_acc: 0.4713

Epoch 00039: val_loss did not improve from 1.23389
Epoch 40/3000
4547/4547 [==============================] - 0s 4us/step - loss: 1.2281 - acc: 0.4794 - val_loss: 1.2326 - val_acc: 0.4728

Epoch 00040: val_loss improved from 1.23389 to 1.23263, saving model to model1/40-1.2326.hdf5
Epoch 41/3000
4547/4

4547/4547 [==============================] - 0s 3us/step - loss: 1.1747 - acc: 0.4942 - val_loss: 1.1873 - val_acc: 0.4903

Epoch 00075: val_loss did not improve from 1.18609
Epoch 76/3000
4547/4547 [==============================] - 0s 3us/step - loss: 1.1655 - acc: 0.5201 - val_loss: 1.1817 - val_acc: 0.4790

Epoch 00076: val_loss improved from 1.18609 to 1.18173, saving model to model1/76-1.1817.hdf5
Epoch 77/3000
4547/4547 [==============================] - 0s 3us/step - loss: 1.1679 - acc: 0.5021 - val_loss: 1.1885 - val_acc: 0.4769

Epoch 00077: val_loss did not improve from 1.18173
Epoch 78/3000
4547/4547 [==============================] - 0s 4us/step - loss: 1.1623 - acc: 0.5082 - val_loss: 1.1801 - val_acc: 0.4979

Epoch 00078: val_loss improved from 1.18173 to 1.18010, saving model to model1/78-1.1801.hdf5
Epoch 79/3000
4547/4547 [==============================] - 0s 3us/step - loss: 1.1561 - acc: 0.5206 - val_loss: 1.1787 - val_acc: 0.4954

Epoch 00079: val_loss improved fro

4547/4547 [==============================] - 0s 3us/step - loss: 1.1283 - acc: 0.5168 - val_loss: 1.1507 - val_acc: 0.4990

Epoch 00116: val_loss improved from 1.15491 to 1.15068, saving model to model1/116-1.1507.hdf5
Epoch 117/3000
4547/4547 [==============================] - 0s 3us/step - loss: 1.1205 - acc: 0.5252 - val_loss: 1.1518 - val_acc: 0.4964

Epoch 00117: val_loss did not improve from 1.15068
Epoch 118/3000
4547/4547 [==============================] - 0s 3us/step - loss: 1.1252 - acc: 0.5166 - val_loss: 1.1517 - val_acc: 0.4938

Epoch 00118: val_loss did not improve from 1.15068
Epoch 119/3000
4547/4547 [==============================] - 0s 3us/step - loss: 1.1422 - acc: 0.5085 - val_loss: 1.1949 - val_acc: 0.4892

Epoch 00119: val_loss did not improve from 1.15068
Epoch 120/3000
4547/4547 [==============================] - 0s 3us/step - loss: 1.1307 - acc: 0.5239 - val_loss: 1.1699 - val_acc: 0.4872

Epoch 00120: val_loss did not improve from 1.15068
Epoch 121/3000
4547/4

...


Epoch 01304: val_loss did not improve from 1.05908
Epoch 1305/3000
4547/4547 [==============================] - 0s 3us/step - loss: 1.0234 - acc: 0.5575 - val_loss: 1.0601 - val_acc: 0.5338

Epoch 01305: val_loss did not improve from 1.05908
Epoch 1306/3000
4547/4547 [==============================] - 0s 3us/step - loss: 1.0178 - acc: 0.5736 - val_loss: 1.0610 - val_acc: 0.5277

Epoch 01306: val_loss did not improve from 1.05908
Epoch 1307/3000
4547/4547 [==============================] - 0s 4us/step - loss: 1.0194 - acc: 0.5615 - val_loss: 1.0637 - val_acc: 0.5328

Epoch 01307: val_loss did not improve from 1.05908
Epoch 1308/3000
4547/4547 [==============================] - 0s 3us/step - loss: 1.0234 - acc: 0.5569 - val_loss: 1.0600 - val_acc: 0.5262

Epoch 01308: val_loss did not improve from 1.05908
Epoch 1309/3000
4547/4547 [==============================] - 0s 3us/step - loss: 1.0206 - acc: 0.5637 - val_loss: 1.0675 - val_acc: 0.5323

Epoch 01309: val_loss did not improve from 1

4547/4547 [==============================] - 0s 3us/step - loss: 1.0200 - acc: 0.5597 - val_loss: 1.0618 - val_acc: 0.5241

Epoch 01347: val_loss did not improve from 1.05885
Epoch 1348/3000
4547/4547 [==============================] - 0s 4us/step - loss: 1.0237 - acc: 0.5626 - val_loss: 1.0706 - val_acc: 0.5215

Epoch 01348: val_loss did not improve from 1.05885
Epoch 1349/3000
4547/4547 [==============================] - 0s 4us/step - loss: 1.0222 - acc: 0.5612 - val_loss: 1.0619 - val_acc: 0.5415

Epoch 01349: val_loss did not improve from 1.05885
Epoch 1350/3000
4547/4547 [==============================] - 0s 3us/step - loss: 1.0161 - acc: 0.5696 - val_loss: 1.0629 - val_acc: 0.5354

Epoch 01350: val_loss did not improve from 1.05885
Epoch 1351/3000
4547/4547 [==============================] - 0s 4us/step - loss: 1.0183 - acc: 0.5626 - val_loss: 1.0626 - val_acc: 0.5369

Epoch 01351: val_loss did not improve from 1.05885
Epoch 1352/3000
4547/4547 [==============================] - 

4547/4547 [==============================] - 0s 3us/step - loss: 1.0196 - acc: 0.5659 - val_loss: 1.0611 - val_acc: 0.5282

Epoch 01389: val_loss did not improve from 1.05782
Epoch 1390/3000
4547/4547 [==============================] - 0s 3us/step - loss: 1.0244 - acc: 0.5601 - val_loss: 1.0629 - val_acc: 0.5379

Epoch 01390: val_loss did not improve from 1.05782
Epoch 1391/3000
4547/4547 [==============================] - 0s 3us/step - loss: 1.0221 - acc: 0.5577 - val_loss: 1.0683 - val_acc: 0.5308

Epoch 01391: val_loss did not improve from 1.05782
Epoch 1392/3000
4547/4547 [==============================] - 0s 3us/step - loss: 1.0230 - acc: 0.5536 - val_loss: 1.0583 - val_acc: 0.5333

Epoch 01392: val_loss did not improve from 1.05782
Epoch 1393/3000
4547/4547 [==============================] - 0s 3us/step - loss: 1.0200 - acc: 0.5632 - val_loss: 1.0595 - val_acc: 0.5364

Epoch 01393: val_loss did not improve from 1.05782
Epoch 1394/3000
4547/4547 [==============================] - 

4547/4547 [==============================] - 0s 3us/step - loss: 1.0190 - acc: 0.5623 - val_loss: 1.0639 - val_acc: 0.5318

Epoch 01432: val_loss did not improve from 1.05782
Epoch 1433/3000
4547/4547 [==============================] - 0s 3us/step - loss: 1.0258 - acc: 0.5604 - val_loss: 1.0602 - val_acc: 0.5338

Epoch 01433: val_loss did not improve from 1.05782
Epoch 1434/3000
4547/4547 [==============================] - 0s 4us/step - loss: 1.0206 - acc: 0.5595 - val_loss: 1.0643 - val_acc: 0.5282

Epoch 01434: val_loss did not improve from 1.05782
Epoch 1435/3000
4547/4547 [==============================] - 0s 3us/step - loss: 1.0232 - acc: 0.5593 - val_loss: 1.0691 - val_acc: 0.5215

Epoch 01435: val_loss did not improve from 1.05782
Epoch 1436/3000
4547/4547 [==============================] - 0s 4us/step - loss: 1.0204 - acc: 0.5648 - val_loss: 1.0647 - val_acc: 0.5256

Epoch 01436: val_loss did not improve from 1.05782
Epoch 1437/3000
4547/4547 [==============================] - 

4547/4547 [==============================] - 0s 3us/step - loss: 1.0188 - acc: 0.5652 - val_loss: 1.0822 - val_acc: 0.5308

Epoch 01475: val_loss did not improve from 1.05782
Epoch 1476/3000
4547/4547 [==============================] - 0s 3us/step - loss: 1.0187 - acc: 0.5641 - val_loss: 1.0587 - val_acc: 0.5410

Epoch 01476: val_loss did not improve from 1.05782
Epoch 1477/3000
4547/4547 [==============================] - 0s 3us/step - loss: 1.0208 - acc: 0.5593 - val_loss: 1.0596 - val_acc: 0.5374

Epoch 01477: val_loss did not improve from 1.05782
Epoch 1478/3000
4547/4547 [==============================] - 0s 3us/step - loss: 1.0145 - acc: 0.5718 - val_loss: 1.0720 - val_acc: 0.5369

Epoch 01478: val_loss did not improve from 1.05782
Epoch 1479/3000
4547/4547 [==============================] - 0s 3us/step - loss: 1.0171 - acc: 0.5711 - val_loss: 1.0600 - val_acc: 0.5385

Epoch 01479: val_loss did not improve from 1.05782
Epoch 1480/3000
4547/4547 [==============================] - 

In [16]:
results = 'result/'
if not os.path.exists(results):
    os.mkdir(results)
# save model
model.save('result/my_model1.h5')
del model
model = load_model('result/my_model1.h5')

# Accuracy result
print("\n Accuracy: %.4f" % (model.evaluate(X, Y)[1]))

del model

6497/6497 [==============================] - 0s 13us/step

 Accuracy: 0.5589


##### model 2 
- input 12
- 4 hidden layer, 30 15 10 * 11 node
- activation function : relu -> tanh -> softplus -> softmax

In [17]:
seed = 0
np.random.seed(seed)
tf.set_random_seed(seed)

# modeling
model = Sequential()
model.add(Dense(30, input_dim=12, activation='relu'))
model.add(Dense(15, activation='tanh'))
model.add(Dense(10, activation='softplus'))
model.add(Dense(11, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [18]:
# model update
MODEL_DIR = 'model2/'
if not os.path.exists(MODEL_DIR):
    os.mkdir(MODEL_DIR)

modelpath = "model2/{epoch:02d}-{val_loss:.4f}.hdf5"
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', verbose=1, save_best_only=True)
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=100)

history2 = model.fit(X, Y, validation_split=0.3, epochs=3000, batch_size=500, callbacks=[early_stopping_callback, checkpointer])

Train on 4547 samples, validate on 1950 samples
Epoch 1/3000
4547/4547 [==============================] - 0s 59us/step - loss: 2.3976 - acc: 0.2248 - val_loss: 2.0647 - val_acc: 0.4118

Epoch 00001: val_loss improved from inf to 2.06468, saving model to model2/01-2.0647.hdf5
Epoch 2/3000
4547/4547 [==============================] - 0s 3us/step - loss: 2.0197 - acc: 0.4256 - val_loss: 1.9373 - val_acc: 0.4436

Epoch 00002: val_loss improved from 2.06468 to 1.93732, saving model to model2/02-1.9373.hdf5
Epoch 3/3000
4547/4547 [==============================] - 0s 3us/step - loss: 1.9043 - acc: 0.4335 - val_loss: 1.8188 - val_acc: 0.4436

Epoch 00003: val_loss improved from 1.93732 to 1.81876, saving model to model2/03-1.8188.hdf5
Epoch 4/3000
4547/4547 [==============================] - 0s 4us/step - loss: 1.7742 - acc: 0.4335 - val_loss: 1.6915 - val_acc: 0.4436

Epoch 00004: val_loss improved from 1.81876 to 1.69153, saving model to model2/04-1.6915.hdf5
Epoch 5/3000
4547/4547 [=======

4547/4547 [==============================] - 0s 3us/step - loss: 1.2804 - acc: 0.4335 - val_loss: 1.2830 - val_acc: 0.4436

Epoch 00036: val_loss did not improve from 1.28264
Epoch 37/3000
4547/4547 [==============================] - 0s 3us/step - loss: 1.2802 - acc: 0.4335 - val_loss: 1.2822 - val_acc: 0.4436

Epoch 00037: val_loss improved from 1.28264 to 1.28224, saving model to model2/37-1.2822.hdf5
Epoch 38/3000
4547/4547 [==============================] - 0s 4us/step - loss: 1.2789 - acc: 0.4335 - val_loss: 1.2805 - val_acc: 0.4436

Epoch 00038: val_loss improved from 1.28224 to 1.28051, saving model to model2/38-1.2805.hdf5
Epoch 39/3000
4547/4547 [==============================] - 0s 4us/step - loss: 1.2783 - acc: 0.4335 - val_loss: 1.2799 - val_acc: 0.4436

Epoch 00039: val_loss improved from 1.28051 to 1.27991, saving model to model2/39-1.2799.hdf5
Epoch 40/3000
4547/4547 [==============================] - 0s 4us/step - loss: 1.2774 - acc: 0.4335 - val_loss: 1.2782 - val_acc:

Epoch 74/3000
4547/4547 [==============================] - 0s 4us/step - loss: 1.1822 - acc: 0.4940 - val_loss: 1.1972 - val_acc: 0.4662

Epoch 00074: val_loss improved from 1.20267 to 1.19725, saving model to model2/74-1.1972.hdf5
Epoch 75/3000
4547/4547 [==============================] - 0s 4us/step - loss: 1.1772 - acc: 0.5005 - val_loss: 1.1932 - val_acc: 0.4656

Epoch 00075: val_loss improved from 1.19725 to 1.19322, saving model to model2/75-1.1932.hdf5
Epoch 76/3000
4547/4547 [==============================] - 0s 5us/step - loss: 1.1746 - acc: 0.4984 - val_loss: 1.1972 - val_acc: 0.4667

Epoch 00076: val_loss did not improve from 1.19322
Epoch 77/3000
4547/4547 [==============================] - 0s 4us/step - loss: 1.1724 - acc: 0.5010 - val_loss: 1.1870 - val_acc: 0.4759

Epoch 00077: val_loss improved from 1.19322 to 1.18699, saving model to model2/77-1.1870.hdf5
Epoch 78/3000
4547/4547 [==============================] - 0s 4us/step - loss: 1.1666 - acc: 0.5111 - val_loss: 1.1


Epoch 00112: val_loss did not improve from 1.14044
Epoch 113/3000
4547/4547 [==============================] - 0s 4us/step - loss: 1.1187 - acc: 0.5146 - val_loss: 1.1437 - val_acc: 0.4856

Epoch 00113: val_loss did not improve from 1.14044
Epoch 114/3000
4547/4547 [==============================] - 0s 4us/step - loss: 1.1075 - acc: 0.5228 - val_loss: 1.1393 - val_acc: 0.4867

Epoch 00114: val_loss improved from 1.14044 to 1.13933, saving model to model2/114-1.1393.hdf5
Epoch 115/3000
4547/4547 [==============================] - 0s 4us/step - loss: 1.1076 - acc: 0.5201 - val_loss: 1.1435 - val_acc: 0.4933

Epoch 00115: val_loss did not improve from 1.13933
Epoch 116/3000
4547/4547 [==============================] - 0s 3us/step - loss: 1.1160 - acc: 0.5144 - val_loss: 1.1399 - val_acc: 0.4903

Epoch 00116: val_loss did not improve from 1.13933
Epoch 117/3000
4547/4547 [==============================] - 0s 4us/step - loss: 1.1097 - acc: 0.5184 - val_loss: 1.1352 - val_acc: 0.4974

Epoch

4547/4547 [==============================] - 0s 4us/step - loss: 1.0932 - acc: 0.5250 - val_loss: 1.1340 - val_acc: 0.4918

Epoch 00154: val_loss did not improve from 1.12738
Epoch 155/3000
4547/4547 [==============================] - 0s 5us/step - loss: 1.0995 - acc: 0.5291 - val_loss: 1.1420 - val_acc: 0.4908

Epoch 00155: val_loss did not improve from 1.12738
Epoch 156/3000
4547/4547 [==============================] - 0s 4us/step - loss: 1.0997 - acc: 0.5225 - val_loss: 1.1347 - val_acc: 0.4949

Epoch 00156: val_loss did not improve from 1.12738
Epoch 157/3000
4547/4547 [==============================] - 0s 4us/step - loss: 1.0984 - acc: 0.5258 - val_loss: 1.1374 - val_acc: 0.4908

Epoch 00157: val_loss did not improve from 1.12738
Epoch 158/3000
4547/4547 [==============================] - 0s 4us/step - loss: 1.1024 - acc: 0.5261 - val_loss: 1.1256 - val_acc: 0.4928

Epoch 00158: val_loss improved from 1.12738 to 1.12560, saving model to model2/158-1.1256.hdf5
Epoch 159/3000
4547/4

...

4547/4547 [==============================] - 0s 4us/step - loss: 1.0493 - acc: 0.5393 - val_loss: 1.0815 - val_acc: 0.5164

Epoch 00535: val_loss did not improve from 1.07927
Epoch 536/3000
4547/4547 [==============================] - 0s 3us/step - loss: 1.0405 - acc: 0.5478 - val_loss: 1.0919 - val_acc: 0.5026

Epoch 00536: val_loss did not improve from 1.07927
Epoch 537/3000
4547/4547 [==============================] - 0s 3us/step - loss: 1.0441 - acc: 0.5454 - val_loss: 1.0841 - val_acc: 0.5133

Epoch 00537: val_loss did not improve from 1.07927
Epoch 538/3000
4547/4547 [==============================] - 0s 4us/step - loss: 1.0385 - acc: 0.5467 - val_loss: 1.0837 - val_acc: 0.5144

Epoch 00538: val_loss did not improve from 1.07927
Epoch 539/3000
4547/4547 [==============================] - 0s 3us/step - loss: 1.0370 - acc: 0.5500 - val_loss: 1.0831 - val_acc: 0.5138

Epoch 00539: val_loss did not improve from 1.07927
Epoch 540/3000
4547/4547 [==============================] - 0s 3u

4547/4547 [==============================] - 0s 4us/step - loss: 1.0364 - acc: 0.5525 - val_loss: 1.1036 - val_acc: 0.4954

Epoch 00578: val_loss did not improve from 1.07883
Epoch 579/3000
4547/4547 [==============================] - 0s 3us/step - loss: 1.0415 - acc: 0.5445 - val_loss: 1.0921 - val_acc: 0.5092

Epoch 00579: val_loss did not improve from 1.07883
Epoch 580/3000
4547/4547 [==============================] - 0s 3us/step - loss: 1.0355 - acc: 0.5566 - val_loss: 1.1056 - val_acc: 0.4964

Epoch 00580: val_loss did not improve from 1.07883
Epoch 581/3000
4547/4547 [==============================] - 0s 3us/step - loss: 1.0466 - acc: 0.5492 - val_loss: 1.0859 - val_acc: 0.5226

Epoch 00581: val_loss did not improve from 1.07883
Epoch 582/3000
4547/4547 [==============================] - 0s 4us/step - loss: 1.0538 - acc: 0.5399 - val_loss: 1.0861 - val_acc: 0.5144

Epoch 00582: val_loss did not improve from 1.07883
Epoch 583/3000
4547/4547 [==============================] - 0s 3u

4547/4547 [==============================] - 0s 3us/step - loss: 1.0351 - acc: 0.5601 - val_loss: 1.0845 - val_acc: 0.5031

Epoch 00621: val_loss did not improve from 1.07847
Epoch 622/3000
4547/4547 [==============================] - 0s 3us/step - loss: 1.0338 - acc: 0.5511 - val_loss: 1.0873 - val_acc: 0.5087

Epoch 00622: val_loss did not improve from 1.07847
Epoch 623/3000
4547/4547 [==============================] - 0s 3us/step - loss: 1.0446 - acc: 0.5529 - val_loss: 1.0883 - val_acc: 0.5159

Epoch 00623: val_loss did not improve from 1.07847
Epoch 624/3000
4547/4547 [==============================] - 0s 3us/step - loss: 1.0346 - acc: 0.5520 - val_loss: 1.0820 - val_acc: 0.5169

Epoch 00624: val_loss did not improve from 1.07847
Epoch 625/3000
4547/4547 [==============================] - 0s 4us/step - loss: 1.0344 - acc: 0.5527 - val_loss: 1.0809 - val_acc: 0.5210

Epoch 00625: val_loss did not improve from 1.07847
Epoch 626/3000
4547/4547 [==============================] - 0s 3u

4547/4547 [==============================] - 0s 3us/step - loss: 1.0352 - acc: 0.5577 - val_loss: 1.0871 - val_acc: 0.5092

Epoch 00664: val_loss did not improve from 1.07814
Epoch 665/3000
4547/4547 [==============================] - 0s 3us/step - loss: 1.0485 - acc: 0.5362 - val_loss: 1.0837 - val_acc: 0.5164

Epoch 00665: val_loss did not improve from 1.07814
Epoch 666/3000
4547/4547 [==============================] - 0s 3us/step - loss: 1.0399 - acc: 0.5465 - val_loss: 1.0827 - val_acc: 0.5154

Epoch 00666: val_loss did not improve from 1.07814
Epoch 667/3000
4547/4547 [==============================] - 0s 3us/step - loss: 1.0381 - acc: 0.5562 - val_loss: 1.0850 - val_acc: 0.5087

Epoch 00667: val_loss did not improve from 1.07814
Epoch 668/3000
4547/4547 [==============================] - 0s 4us/step - loss: 1.0370 - acc: 0.5564 - val_loss: 1.0927 - val_acc: 0.5159

Epoch 00668: val_loss did not improve from 1.07814
Epoch 669/3000
4547/4547 [==============================] - 0s 4u

4547/4547 [==============================] - 0s 4us/step - loss: 1.0311 - acc: 0.5584 - val_loss: 1.0850 - val_acc: 0.5026

Epoch 00707: val_loss did not improve from 1.07814
Epoch 708/3000
4547/4547 [==============================] - 0s 4us/step - loss: 1.0457 - acc: 0.5412 - val_loss: 1.0946 - val_acc: 0.4979

Epoch 00708: val_loss did not improve from 1.07814
Epoch 709/3000
4547/4547 [==============================] - 0s 4us/step - loss: 1.0399 - acc: 0.5472 - val_loss: 1.0788 - val_acc: 0.5169

Epoch 00709: val_loss did not improve from 1.07814
Epoch 710/3000
4547/4547 [==============================] - 0s 4us/step - loss: 1.0365 - acc: 0.5536 - val_loss: 1.0951 - val_acc: 0.5046

Epoch 00710: val_loss did not improve from 1.07814
Epoch 711/3000
4547/4547 [==============================] - 0s 4us/step - loss: 1.0328 - acc: 0.5500 - val_loss: 1.0808 - val_acc: 0.5154

Epoch 00711: val_loss did not improve from 1.07814
Epoch 712/3000
4547/4547 [==============================] - 0s 4u

In [19]:
# save model
model.save('result/my_model2.h5')
del model
model = load_model('result/my_model2.h5')

# Accuracy result
print("\n Accuracy: %.4f" % (model.evaluate(X, Y)[1]))

del model

6497/6497 [==============================] - 0s 18us/step

 Accuracy: 0.5339


##### model 3
- input 12
- 4 hidden layer , 30 10 10 * 11 node
- activation function : relu -> softplus -> softplus -> softmax

In [20]:
seed = 0
np.random.seed(seed)
tf.set_random_seed(seed)

# modeling
model = Sequential()
model.add(Dense(30, input_dim=12, activation='relu'))
model.add(Dense(10, activation='softplus'))
model.add(Dense(10, activation='softplus'))
model.add(Dense(11, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [21]:
# model update
MODEL_DIR = 'model3/'
if not os.path.exists(MODEL_DIR):
    os.mkdir(MODEL_DIR)

modelpath = "model3/{epoch:02d}-{val_loss:.4f}.hdf5"
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', verbose=1, save_best_only=True)
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=100)

history3 = model.fit(X, Y, validation_split=0.3, epochs=3000, batch_size=500, callbacks=[early_stopping_callback, checkpointer])

Train on 4547 samples, validate on 1950 samples
Epoch 1/3000
4547/4547 [==============================] - 0s 75us/step - loss: 11.8390 - acc: 0.1638 - val_loss: 11.0235 - val_acc: 0.1713

Epoch 00001: val_loss improved from inf to 11.02348, saving model to model3/01-11.0235.hdf5
Epoch 2/3000
4547/4547 [==============================] - 0s 4us/step - loss: 9.8517 - acc: 0.1638 - val_loss: 7.2235 - val_acc: 0.1713

Epoch 00002: val_loss improved from 11.02348 to 7.22347, saving model to model3/02-7.2235.hdf5
Epoch 3/3000
4547/4547 [==============================] - 0s 4us/step - loss: 4.4287 - acc: 0.1990 - val_loss: 1.7706 - val_acc: 0.4328

Epoch 00003: val_loss improved from 7.22347 to 1.77059, saving model to model3/03-1.7706.hdf5
Epoch 4/3000
4547/4547 [==============================] - 0s 6us/step - loss: 2.1055 - acc: 0.4315 - val_loss: 2.2301 - val_acc: 0.4477

Epoch 00004: val_loss did not improve from 1.77059
Epoch 5/3000
4547/4547 [==============================] - 0s 4us/step


Epoch 00036: val_loss improved from 1.23317 to 1.23133, saving model to model3/36-1.2313.hdf5
Epoch 37/3000
4547/4547 [==============================] - 0s 4us/step - loss: 1.2271 - acc: 0.4693 - val_loss: 1.2288 - val_acc: 0.4585

Epoch 00037: val_loss improved from 1.23133 to 1.22876, saving model to model3/37-1.2288.hdf5
Epoch 38/3000
4547/4547 [==============================] - 0s 4us/step - loss: 1.2244 - acc: 0.4682 - val_loss: 1.2263 - val_acc: 0.4605

Epoch 00038: val_loss improved from 1.22876 to 1.22634, saving model to model3/38-1.2263.hdf5
Epoch 39/3000
4547/4547 [==============================] - 0s 4us/step - loss: 1.2213 - acc: 0.4621 - val_loss: 1.2257 - val_acc: 0.4610

Epoch 00039: val_loss improved from 1.22634 to 1.22565, saving model to model3/39-1.2257.hdf5
Epoch 40/3000
4547/4547 [==============================] - 0s 4us/step - loss: 1.2170 - acc: 0.4764 - val_loss: 1.2229 - val_acc: 0.4533

Epoch 00040: val_loss improved from 1.22565 to 1.22292, saving model to

4547/4547 [==============================] - 0s 4us/step - loss: 1.1392 - acc: 0.5135 - val_loss: 1.1821 - val_acc: 0.4692

Epoch 00074: val_loss did not improve from 1.14131
Epoch 75/3000
4547/4547 [==============================] - 0s 3us/step - loss: 1.1528 - acc: 0.5067 - val_loss: 1.1547 - val_acc: 0.4821

Epoch 00075: val_loss did not improve from 1.14131
Epoch 76/3000
4547/4547 [==============================] - 0s 4us/step - loss: 1.1372 - acc: 0.5100 - val_loss: 1.1514 - val_acc: 0.4728

Epoch 00076: val_loss did not improve from 1.14131
Epoch 77/3000
4547/4547 [==============================] - 0s 3us/step - loss: 1.1325 - acc: 0.5166 - val_loss: 1.1425 - val_acc: 0.4764

Epoch 00077: val_loss did not improve from 1.14131
Epoch 78/3000
4547/4547 [==============================] - 0s 3us/step - loss: 1.1201 - acc: 0.5188 - val_loss: 1.1367 - val_acc: 0.4872

Epoch 00078: val_loss improved from 1.14131 to 1.13671, saving model to model3/78-1.1367.hdf5
Epoch 79/3000
4547/4547 [=

4547/4547 [==============================] - 0s 3us/step - loss: 1.0894 - acc: 0.5338 - val_loss: 1.1231 - val_acc: 0.4979

Epoch 00115: val_loss did not improve from 1.11680
Epoch 116/3000
4547/4547 [==============================] - 0s 3us/step - loss: 1.0909 - acc: 0.5333 - val_loss: 1.1162 - val_acc: 0.5169

Epoch 00116: val_loss improved from 1.11680 to 1.11624, saving model to model3/116-1.1162.hdf5
Epoch 117/3000
4547/4547 [==============================] - 0s 4us/step - loss: 1.0881 - acc: 0.5305 - val_loss: 1.1172 - val_acc: 0.5031

Epoch 00117: val_loss did not improve from 1.11624
Epoch 118/3000
4547/4547 [==============================] - 0s 3us/step - loss: 1.1003 - acc: 0.5245 - val_loss: 1.1635 - val_acc: 0.4964

Epoch 00118: val_loss did not improve from 1.11624
Epoch 119/3000
4547/4547 [==============================] - 0s 3us/step - loss: 1.1405 - acc: 0.5087 - val_loss: 1.1712 - val_acc: 0.4851

Epoch 00119: val_loss did not improve from 1.11624
Epoch 120/3000
4547/4

4547/4547 [==============================] - 0s 3us/step - loss: 1.0761 - acc: 0.5382 - val_loss: 1.1577 - val_acc: 0.5026

Epoch 00156: val_loss did not improve from 1.09760
Epoch 157/3000
4547/4547 [==============================] - 0s 4us/step - loss: 1.0845 - acc: 0.5393 - val_loss: 1.1028 - val_acc: 0.5118

Epoch 00157: val_loss did not improve from 1.09760
Epoch 158/3000
4547/4547 [==============================] - 0s 4us/step - loss: 1.0686 - acc: 0.5478 - val_loss: 1.1414 - val_acc: 0.4897

Epoch 00158: val_loss did not improve from 1.09760
Epoch 159/3000
4547/4547 [==============================] - 0s 3us/step - loss: 1.1110 - acc: 0.5206 - val_loss: 1.1001 - val_acc: 0.5103

Epoch 00159: val_loss did not improve from 1.09760
Epoch 160/3000
4547/4547 [==============================] - 0s 4us/step - loss: 1.0731 - acc: 0.5377 - val_loss: 1.1409 - val_acc: 0.4969

Epoch 00160: val_loss did not improve from 1.09760
Epoch 161/3000
4547/4547 [==============================] - 0s 4u

...

4547/4547 [==============================] - 0s 3us/step - loss: 1.0347 - acc: 0.5507 - val_loss: 1.0866 - val_acc: 0.5231

Epoch 00367: val_loss did not improve from 1.08071
Epoch 368/3000
4547/4547 [==============================] - 0s 4us/step - loss: 1.0379 - acc: 0.5544 - val_loss: 1.0962 - val_acc: 0.5103

Epoch 00368: val_loss did not improve from 1.08071
Epoch 369/3000
4547/4547 [==============================] - 0s 4us/step - loss: 1.0580 - acc: 0.5417 - val_loss: 1.0884 - val_acc: 0.5087

Epoch 00369: val_loss did not improve from 1.08071
Epoch 370/3000
4547/4547 [==============================] - 0s 4us/step - loss: 1.0460 - acc: 0.5463 - val_loss: 1.0874 - val_acc: 0.5287

Epoch 00370: val_loss did not improve from 1.08071
Epoch 371/3000
4547/4547 [==============================] - 0s 3us/step - loss: 1.0369 - acc: 0.5474 - val_loss: 1.0838 - val_acc: 0.5190

Epoch 00371: val_loss did not improve from 1.08071
Epoch 372/3000
4547/4547 [==============================] - 0s 4u

4547/4547 [==============================] - 0s 3us/step - loss: 1.0322 - acc: 0.5492 - val_loss: 1.0868 - val_acc: 0.5251

Epoch 00409: val_loss did not improve from 1.07898
Epoch 410/3000
4547/4547 [==============================] - 0s 3us/step - loss: 1.0319 - acc: 0.5560 - val_loss: 1.0817 - val_acc: 0.5133

Epoch 00410: val_loss did not improve from 1.07898
Epoch 411/3000
4547/4547 [==============================] - 0s 4us/step - loss: 1.0350 - acc: 0.5472 - val_loss: 1.0925 - val_acc: 0.5154

Epoch 00411: val_loss did not improve from 1.07898
Epoch 412/3000
4547/4547 [==============================] - 0s 4us/step - loss: 1.0357 - acc: 0.5577 - val_loss: 1.0848 - val_acc: 0.5092

Epoch 00412: val_loss did not improve from 1.07898
Epoch 413/3000
4547/4547 [==============================] - 0s 3us/step - loss: 1.0337 - acc: 0.5531 - val_loss: 1.0803 - val_acc: 0.5159

Epoch 00413: val_loss did not improve from 1.07898
Epoch 414/3000
4547/4547 [==============================] - 0s 4u

4547/4547 [==============================] - 0s 4us/step - loss: 1.0304 - acc: 0.5593 - val_loss: 1.1169 - val_acc: 0.5215

Epoch 00452: val_loss did not improve from 1.07764
Epoch 453/3000
4547/4547 [==============================] - 0s 4us/step - loss: 1.0363 - acc: 0.5489 - val_loss: 1.0854 - val_acc: 0.5133

Epoch 00453: val_loss did not improve from 1.07764
Epoch 454/3000
4547/4547 [==============================] - 0s 3us/step - loss: 1.0387 - acc: 0.5551 - val_loss: 1.0771 - val_acc: 0.5241

Epoch 00454: val_loss improved from 1.07764 to 1.07708, saving model to model3/454-1.0771.hdf5
Epoch 455/3000
4547/4547 [==============================] - 0s 4us/step - loss: 1.0298 - acc: 0.5518 - val_loss: 1.1094 - val_acc: 0.5144

Epoch 00455: val_loss did not improve from 1.07708
Epoch 456/3000
4547/4547 [==============================] - 0s 4us/step - loss: 1.0356 - acc: 0.5509 - val_loss: 1.0978 - val_acc: 0.5236

Epoch 00456: val_loss did not improve from 1.07708
Epoch 457/3000
4547/4

4547/4547 [==============================] - 0s 4us/step - loss: 1.0337 - acc: 0.5531 - val_loss: 1.0895 - val_acc: 0.5251

Epoch 00494: val_loss did not improve from 1.07428
Epoch 495/3000
4547/4547 [==============================] - 0s 4us/step - loss: 1.0365 - acc: 0.5483 - val_loss: 1.0751 - val_acc: 0.5282

Epoch 00495: val_loss did not improve from 1.07428
Epoch 496/3000
4547/4547 [==============================] - 0s 4us/step - loss: 1.0270 - acc: 0.5586 - val_loss: 1.0815 - val_acc: 0.5292

Epoch 00496: val_loss did not improve from 1.07428
Epoch 497/3000
4547/4547 [==============================] - 0s 4us/step - loss: 1.0243 - acc: 0.5555 - val_loss: 1.0806 - val_acc: 0.5262

Epoch 00497: val_loss did not improve from 1.07428
Epoch 498/3000
4547/4547 [==============================] - 0s 4us/step - loss: 1.0216 - acc: 0.5621 - val_loss: 1.0952 - val_acc: 0.5097

Epoch 00498: val_loss did not improve from 1.07428
Epoch 499/3000
4547/4547 [==============================] - 0s 4u

4547/4547 [==============================] - 0s 3us/step - loss: 1.0296 - acc: 0.5582 - val_loss: 1.0853 - val_acc: 0.5236

Epoch 00537: val_loss did not improve from 1.07428
Epoch 538/3000
4547/4547 [==============================] - 0s 3us/step - loss: 1.0260 - acc: 0.5621 - val_loss: 1.0803 - val_acc: 0.5277

Epoch 00538: val_loss did not improve from 1.07428
Epoch 539/3000
4547/4547 [==============================] - 0s 4us/step - loss: 1.0227 - acc: 0.5560 - val_loss: 1.0807 - val_acc: 0.5241

Epoch 00539: val_loss did not improve from 1.07428
Epoch 540/3000
4547/4547 [==============================] - 0s 3us/step - loss: 1.0223 - acc: 0.5595 - val_loss: 1.0776 - val_acc: 0.5174

Epoch 00540: val_loss did not improve from 1.07428
Epoch 541/3000
4547/4547 [==============================] - 0s 3us/step - loss: 1.0248 - acc: 0.5582 - val_loss: 1.0748 - val_acc: 0.5323

Epoch 00541: val_loss did not improve from 1.07428
Epoch 542/3000
4547/4547 [==============================] - 0s 3u

In [22]:
# save model
model.save('result/my_model3.h5')
del model
model = load_model('result/my_model3.h5')

# Accuracy result
print("\n Accuracy: %.4f" % (model.evaluate(X, Y)[1]))

del model

6497/6497 [==============================] - 0s 23us/step

 Accuracy: 0.5490


##### model 4 
- input 12
- 4 hidden layer 30 10 10 * 11 node
- activation function : softplus -> softplus -> softplus -> softmax

In [23]:
seed = 0
np.random.seed(seed)
tf.set_random_seed(seed)

# modeling
model = Sequential()
model.add(Dense(30, input_dim=12, activation='softplus'))
model.add(Dense(10, activation='softplus'))
model.add(Dense(10, activation='softplus'))
model.add(Dense(11, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [24]:
# model update
MODEL_DIR = 'model4/'
if not os.path.exists(MODEL_DIR):
    os.mkdir(MODEL_DIR)

modelpath = "model4/{epoch:02d}-{val_loss:.4f}.hdf5"
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', verbose=1, save_best_only=True)
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=100)

history4 = model.fit(X, Y, validation_split=0.3, epochs=3000, batch_size=500, callbacks=[early_stopping_callback, checkpointer])

Train on 4547 samples, validate on 1950 samples
Epoch 1/3000
4547/4547 [==============================] - 0s 90us/step - loss: 11.8432 - acc: 0.1638 - val_loss: 11.0262 - val_acc: 0.1713

Epoch 00001: val_loss improved from inf to 11.02625, saving model to model4/01-11.0262.hdf5
Epoch 2/3000
4547/4547 [==============================] - 0s 6us/step - loss: 9.8499 - acc: 0.1638 - val_loss: 7.2071 - val_acc: 0.1713

Epoch 00002: val_loss improved from 11.02625 to 7.20706, saving model to model4/02-7.2071.hdf5
Epoch 3/3000
4547/4547 [==============================] - 0s 4us/step - loss: 4.3982 - acc: 0.2012 - val_loss: 1.7834 - val_acc: 0.4318

Epoch 00003: val_loss improved from 7.20706 to 1.78344, saving model to model4/03-1.7834.hdf5
Epoch 4/3000
4547/4547 [==============================] - 0s 4us/step - loss: 2.1149 - acc: 0.4313 - val_loss: 2.2296 - val_acc: 0.4482

Epoch 00004: val_loss did not improve from 1.78344
Epoch 5/3000
4547/4547 [==============================] - 0s 4us/step


Epoch 00036: val_loss improved from 1.20878 to 1.20229, saving model to model4/36-1.2023.hdf5
Epoch 37/3000
4547/4547 [==============================] - 0s 4us/step - loss: 1.2032 - acc: 0.4858 - val_loss: 1.2008 - val_acc: 0.4605

Epoch 00037: val_loss improved from 1.20229 to 1.20077, saving model to model4/37-1.2008.hdf5
Epoch 38/3000
4547/4547 [==============================] - 0s 4us/step - loss: 1.1994 - acc: 0.4803 - val_loss: 1.1980 - val_acc: 0.4677

Epoch 00038: val_loss improved from 1.20077 to 1.19796, saving model to model4/38-1.1980.hdf5
Epoch 39/3000
4547/4547 [==============================] - 0s 4us/step - loss: 1.1956 - acc: 0.4733 - val_loss: 1.2001 - val_acc: 0.4626

Epoch 00039: val_loss did not improve from 1.19796
Epoch 40/3000
4547/4547 [==============================] - 0s 4us/step - loss: 1.1906 - acc: 0.4788 - val_loss: 1.1936 - val_acc: 0.4590

Epoch 00040: val_loss improved from 1.19796 to 1.19362, saving model to model4/40-1.1936.hdf5
Epoch 41/3000
4547/4

4547/4547 [==============================] - 0s 4us/step - loss: 1.1327 - acc: 0.5113 - val_loss: 1.1420 - val_acc: 0.4908

Epoch 00075: val_loss did not improve from 1.13412
Epoch 76/3000
4547/4547 [==============================] - 0s 4us/step - loss: 1.1192 - acc: 0.5219 - val_loss: 1.1422 - val_acc: 0.4764

Epoch 00076: val_loss did not improve from 1.13412
Epoch 77/3000
4547/4547 [==============================] - 0s 5us/step - loss: 1.1195 - acc: 0.5144 - val_loss: 1.1286 - val_acc: 0.4903

Epoch 00077: val_loss improved from 1.13412 to 1.12858, saving model to model4/77-1.1286.hdf5
Epoch 78/3000
4547/4547 [==============================] - 0s 5us/step - loss: 1.1052 - acc: 0.5234 - val_loss: 1.1255 - val_acc: 0.4908

Epoch 00078: val_loss improved from 1.12858 to 1.12545, saving model to model4/78-1.1255.hdf5
Epoch 79/3000
4547/4547 [==============================] - 0s 4us/step - loss: 1.1058 - acc: 0.5221 - val_loss: 1.1249 - val_acc: 0.4954

Epoch 00079: val_loss improved fro


Epoch 00115: val_loss did not improve from 1.10764
Epoch 116/3000
4547/4547 [==============================] - 0s 4us/step - loss: 1.0811 - acc: 0.5355 - val_loss: 1.1039 - val_acc: 0.5097

Epoch 00116: val_loss improved from 1.10764 to 1.10394, saving model to model4/116-1.1039.hdf5
Epoch 117/3000
4547/4547 [==============================] - 0s 4us/step - loss: 1.0771 - acc: 0.5291 - val_loss: 1.1073 - val_acc: 0.4979

Epoch 00117: val_loss did not improve from 1.10394
Epoch 118/3000
4547/4547 [==============================] - 0s 4us/step - loss: 1.0891 - acc: 0.5274 - val_loss: 1.1374 - val_acc: 0.4985

Epoch 00118: val_loss did not improve from 1.10394
Epoch 119/3000
4547/4547 [==============================] - 0s 4us/step - loss: 1.1205 - acc: 0.5120 - val_loss: 1.1597 - val_acc: 0.4944

Epoch 00119: val_loss did not improve from 1.10394
Epoch 120/3000
4547/4547 [==============================] - 0s 4us/step - loss: 1.0986 - acc: 0.5133 - val_loss: 1.1068 - val_acc: 0.5051

Epoch

4547/4547 [==============================] - 0s 4us/step - loss: 1.0731 - acc: 0.5388 - val_loss: 1.1133 - val_acc: 0.5056

Epoch 00157: val_loss did not improve from 1.09105
Epoch 158/3000
4547/4547 [==============================] - 0s 5us/step - loss: 1.0679 - acc: 0.5423 - val_loss: 1.1292 - val_acc: 0.4923

Epoch 00158: val_loss did not improve from 1.09105
Epoch 159/3000
4547/4547 [==============================] - 0s 4us/step - loss: 1.1010 - acc: 0.5236 - val_loss: 1.1062 - val_acc: 0.5031

Epoch 00159: val_loss did not improve from 1.09105
Epoch 160/3000
4547/4547 [==============================] - 0s 4us/step - loss: 1.0651 - acc: 0.5423 - val_loss: 1.1155 - val_acc: 0.4979

Epoch 00160: val_loss did not improve from 1.09105
Epoch 161/3000
4547/4547 [==============================] - 0s 4us/step - loss: 1.0652 - acc: 0.5426 - val_loss: 1.0956 - val_acc: 0.5149

Epoch 00161: val_loss did not improve from 1.09105
Epoch 162/3000
4547/4547 [==============================] - 0s 4u

...

4547/4547 [==============================] - 0s 4us/step - loss: 1.0301 - acc: 0.5538 - val_loss: 1.0803 - val_acc: 0.5092

Epoch 00368: val_loss did not improve from 1.07206
Epoch 369/3000
4547/4547 [==============================] - 0s 5us/step - loss: 1.0445 - acc: 0.5505 - val_loss: 1.0924 - val_acc: 0.5056

Epoch 00369: val_loss did not improve from 1.07206
Epoch 370/3000
4547/4547 [==============================] - 0s 5us/step - loss: 1.0353 - acc: 0.5533 - val_loss: 1.0760 - val_acc: 0.5154

Epoch 00370: val_loss did not improve from 1.07206
Epoch 371/3000
4547/4547 [==============================] - 0s 6us/step - loss: 1.0276 - acc: 0.5560 - val_loss: 1.0735 - val_acc: 0.5205

Epoch 00371: val_loss did not improve from 1.07206
Epoch 372/3000
4547/4547 [==============================] - 0s 5us/step - loss: 1.0246 - acc: 0.5564 - val_loss: 1.0713 - val_acc: 0.5154

Epoch 00372: val_loss improved from 1.07206 to 1.07135, saving model to model4/372-1.0713.hdf5
Epoch 373/3000
4547/4

4547/4547 [==============================] - 0s 5us/step - loss: 1.0238 - acc: 0.5623 - val_loss: 1.0967 - val_acc: 0.5087

Epoch 00411: val_loss did not improve from 1.07058
Epoch 412/3000
4547/4547 [==============================] - 0s 4us/step - loss: 1.0333 - acc: 0.5604 - val_loss: 1.0988 - val_acc: 0.5133

Epoch 00412: val_loss did not improve from 1.07058
Epoch 413/3000
4547/4547 [==============================] - 0s 4us/step - loss: 1.0363 - acc: 0.5509 - val_loss: 1.0733 - val_acc: 0.5159

Epoch 00413: val_loss did not improve from 1.07058
Epoch 414/3000
4547/4547 [==============================] - 0s 5us/step - loss: 1.0247 - acc: 0.5593 - val_loss: 1.0740 - val_acc: 0.5210

Epoch 00414: val_loss did not improve from 1.07058
Epoch 415/3000
4547/4547 [==============================] - 0s 4us/step - loss: 1.0275 - acc: 0.5566 - val_loss: 1.0769 - val_acc: 0.5200

Epoch 00415: val_loss did not improve from 1.07058
Epoch 416/3000
4547/4547 [==============================] - 0s 4u

4547/4547 [==============================] - 0s 4us/step - loss: 1.0281 - acc: 0.5612 - val_loss: 1.0721 - val_acc: 0.5210

Epoch 00454: val_loss did not improve from 1.07058
Epoch 455/3000
4547/4547 [==============================] - 0s 4us/step - loss: 1.0194 - acc: 0.5639 - val_loss: 1.0839 - val_acc: 0.5256

Epoch 00455: val_loss did not improve from 1.07058
Epoch 456/3000
4547/4547 [==============================] - 0s 4us/step - loss: 1.0220 - acc: 0.5601 - val_loss: 1.0975 - val_acc: 0.5231

Epoch 00456: val_loss did not improve from 1.07058
Epoch 457/3000
4547/4547 [==============================] - 0s 5us/step - loss: 1.0205 - acc: 0.5617 - val_loss: 1.0910 - val_acc: 0.5200

Epoch 00457: val_loss did not improve from 1.07058
Epoch 458/3000
4547/4547 [==============================] - 0s 5us/step - loss: 1.0269 - acc: 0.5610 - val_loss: 1.0738 - val_acc: 0.5287

Epoch 00458: val_loss did not improve from 1.07058
Epoch 459/3000
4547/4547 [==============================] - 0s 5u

4547/4547 [==============================] - 0s 4us/step - loss: 1.0167 - acc: 0.5663 - val_loss: 1.0714 - val_acc: 0.5241

Epoch 00497: val_loss did not improve from 1.06992
Epoch 498/3000
4547/4547 [==============================] - 0s 4us/step - loss: 1.0144 - acc: 0.5681 - val_loss: 1.0831 - val_acc: 0.5046

Epoch 00498: val_loss did not improve from 1.06992
Epoch 499/3000
4547/4547 [==============================] - 0s 4us/step - loss: 1.0300 - acc: 0.5599 - val_loss: 1.0882 - val_acc: 0.5159

Epoch 00499: val_loss did not improve from 1.06992
Epoch 500/3000
4547/4547 [==============================] - 0s 4us/step - loss: 1.0253 - acc: 0.5558 - val_loss: 1.1004 - val_acc: 0.5185

Epoch 00500: val_loss did not improve from 1.06992
Epoch 501/3000
4547/4547 [==============================] - 0s 4us/step - loss: 1.0262 - acc: 0.5630 - val_loss: 1.0773 - val_acc: 0.5262

Epoch 00501: val_loss did not improve from 1.06992
Epoch 502/3000
4547/4547 [==============================] - 0s 4u

4547/4547 [==============================] - 0s 4us/step - loss: 1.0156 - acc: 0.5621 - val_loss: 1.0721 - val_acc: 0.5169

Epoch 00540: val_loss did not improve from 1.06992
Epoch 541/3000
4547/4547 [==============================] - 0s 4us/step - loss: 1.0199 - acc: 0.5670 - val_loss: 1.0716 - val_acc: 0.5123

Epoch 00541: val_loss did not improve from 1.06992
Epoch 542/3000
4547/4547 [==============================] - 0s 4us/step - loss: 1.0165 - acc: 0.5683 - val_loss: 1.0768 - val_acc: 0.5236

Epoch 00542: val_loss did not improve from 1.06992
Epoch 543/3000
4547/4547 [==============================] - 0s 4us/step - loss: 1.0183 - acc: 0.5626 - val_loss: 1.0730 - val_acc: 0.5169

Epoch 00543: val_loss did not improve from 1.06992
Epoch 544/3000
4547/4547 [==============================] - 0s 4us/step - loss: 1.0186 - acc: 0.5630 - val_loss: 1.0727 - val_acc: 0.5226

Epoch 00544: val_loss did not improve from 1.06992
Epoch 545/3000
4547/4547 [==============================] - 0s 4u

In [25]:
# save model
model.save('result/my_model4.h5')
del model
model = load_model('result/my_model4.h5')

# Accuracy result
print("\n Accuracy: %.4f" % (model.evaluate(X, Y)[1]))

del model

6497/6497 [==============================] - 0s 28us/step

 Accuracy: 0.5584
